In [46]:
import pandas as pd
import glob
import datetime

In [3]:
jhu_data_path=r'C:\Users\hites\Downloads\COVID-19-master\csse_covid_19_data\csse_covid_19_daily_reports_us'

In [40]:
ny_times_path=r'C:\Users\hites\Downloads\nytimes\us-states.csv'

In [6]:
jhu_files=glob.glob(jhu_data_path+"/*.csv")

In [101]:
li=[]
date=[]

for filename in jhu_files:
    df=pd.read_csv(filename, header=0)
    date.append((filename.split('\\')[-1].split('.')[0]))
    li.append(df)

jhu_df=pd.concat(li, axis=0)

C:\Users\hites\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [102]:
date_final=[d for d in date for i in range(58)]
jhu_df['date']=date_final

In [103]:
jhu_df['date']=pd.to_datetime(jhu_df['date'])



In [104]:
ny_times_df=pd.read_csv(ny_times_path)

In [105]:
ny_times_df['date']=pd.to_datetime(ny_times_df['date'])

In [106]:
start_date=datetime.datetime.strptime('2020-04-30', '%Y-%m-%d')
end_date=datetime.datetime.strptime('2021-01-01', '%Y-%m-%d')
ny_times_df=ny_times_df[ny_times_df.date>start_date]  #Data from 1st May, 2020
ny_times_df=ny_times_df[ny_times_df.date<end_date]

In [107]:
jhu_df=jhu_df[jhu_df.date>start_date] #Data from 1st May, 2020
jhu_df=jhu_df[jhu_df.date<end_date]

In [109]:
jhu_df=jhu_df[jhu_df.ISO3=='USA']

In [110]:
jhu_states=list(jhu_df.Province_State)
ny_times_states=list(ny_times_df.state)

In [117]:
matched_states=list(set([state for state in ny_times_states if state in jhu_states]))

In [124]:
final_jhu=jhu_df[jhu_df['Province_State'].isin(matched_states)]
final_nytimes=ny_times_df[ny_times_df['state'].isin(matched_states)]

In [127]:
final_jhu.head()

,Active,Case_Fatality_Ratio,Confirmed,Country_Region,Deaths,FIPS,Hospitalization_Rate,ISO3,Incident_Rate,Last_Update,...,Long_,Mortality_Rate,People_Hospitalized,People_Tested,Province_State,Recovered,Testing_Rate,Total_Test_Results,UID,date
0,7005.0,NaN,7440,US,289,1.0,13.819578,USA,155.558973,2020-05-02 02:32:31,...,-86.9023,3.962161,1008.0,91933.0,Alabama,0.0,1960.653006,NaN,84000001.0,2020-05-01
1,101.0,NaN,364,US,9,2.0,NaN,USA,60.895125,2020-05-02 02:32:31,...,-152.4044,2.472527,NaN,20325.0,Alaska,254.0,3400.256629,NaN,84000002.0,2020-05-01
3,6111.0,NaN,7969,US,330,4.0,15.095997,USA,109.483581,2020-05-02 02:32:31,...,-111.4312,4.141047,1203.0,74879.0,Arizona,1528.0,1028.738993,NaN,84000004.0,2020-05-01
4,1300.0,NaN,3337,US,64,5.0,12.406353,USA,128.890120,2020-05-02 02:32:31,...,-92.3731,1.917890,414.0,49676.0,Arkansas,1973.0,1918.713092,NaN,84000005.0,2020-05-01
5,49900.0,NaN,52026,US,2126,6.0,NaN,USA,132.690209,2020-05-02 02:32:31,...,-119.6816,4.086418,NaN,654985.0,California,0.0,1670.512760,NaN,84000006.0,2020-05-01


In [155]:
def get_cases(state,date):
    df=final_nytimes[final_nytimes.date==date]
    
    cases=df[df.state==state]
    
    c=int(cases['cases'])
    
    return c

In [156]:
merged_df=final_jhu.apply(lambda row: get_cases(row['Province_State'], row['date']), axis=1)

In [159]:
merged_df=list(merged_df)

In [161]:
final_jhu['ny_times_cases']=merged_df

C:\Users\hites\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [165]:
final_jhu['difference']=final_jhu['Confirmed']-final_jhu['ny_times_cases']

C:\Users\hites\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [169]:
average_diff=sum(final_jhu['difference'])/len(final_jhu)

In [170]:
average_diff

-132.92124849939975